In [0]:
from pyspark.sql import functions as F

In [0]:
spark.conf.set("spark.sql.session.timeZone", "America/Sao_Paulo")

In [0]:
origem_path = '/Volumes/fomacao_microsoft_power_bi_profisional/bronze/landing/processados/pessoas.csv'

In [0]:
df_raw = (spark.read
          .option('header', 'true')
          .option('enconding', 'UTF-8')
          .option('sep', ',')
          .option('inferSchema', 'true')
          .option('multiline', 'true')
          .option('mode', 'PERMISSIVE')
          .csv(origem_path)
          .withColumn('_ingest_ts_utc', F.current_timestamp())
          .withColumn("_ingest_date", F.to_date(F.col("_ingest_ts_utc")))
          .withColumn("_source_path", F.col("_metadata.file_path"))
          .withColumn("_source_file", F.regexp_extract(F.col("_source_path"), r"([^/]+)$", 1)))

In [0]:
tbl = 'pessoas'
destino_path = f'fomacao_microsoft_power_bi_profisional.bronze.{tbl}'

df_bronze = (df_raw.write
             .format('delta')
             .mode('overwrite')
             .option('overwriteSchema', 'true')
             .partitionBy('_ingest_date')
             .saveAsTable(destino_path))

print(f'Contagem de registros: {spark.table(destino_path).count()}')